In [1]:
# Cell 1: Imports
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.data.data_loader import DataLoader, load_sample_data
from src.data.data_preprocessing import DataPreprocessor
from src.features.feature_builder import FeatureBuilder
from src.models.train import ModelTrainer
from src.utils.helpers import setup_logging, set_random_seed

setup_logging(log_level="INFO")
set_random_seed(42)
%matplotlib inline

print("Imports completed!")

2025-11-27 15:22:30 | INFO     | helpers:set_random_seed:241 - Random seed set to 42


Imports completed!


In [2]:
# Cell 2: Prepare Data
# Load and preprocess data
data_loader = DataLoader()
try:
    df = data_loader.load_csv()
except FileNotFoundError:
    df = load_sample_data()

preprocessor = DataPreprocessor()
df_clean = preprocessor.clean_data(df)
df_features = preprocessor.create_features(df_clean)

# Split data
train_df, test_df = train_test_split(df_features, test_size=0.2, random_state=42)

# Prepare features
X_train, y_train = preprocessor.prepare_features(train_df, fit=True)
X_test, y_test = preprocessor.prepare_features(test_df, fit=False)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

2025-11-27 15:22:33 | INFO     | config_loader:__init__:47 - Configuration loaded from D:\practice-1\src\config\config.yaml
2025-11-27 15:22:33 | INFO     | data_loader:__init__:62 - DataLoader initialized
2025-11-27 15:22:33 | INFO     | data_loader:load_sample_data:333 - Created sample data with 10 rows
2025-11-27 15:22:33 | INFO     | data_preprocessing:__init__:46 - DataPreprocessor initialized
2025-11-27 15:22:33 | INFO     | data_preprocessing:clean_data:59 - Starting data cleaning. Initial shape: (10, 11)
2025-11-27 15:22:33 | INFO     | data_preprocessing:_remove_outliers:236 - Removed 2 outlier rows (20.00%)
2025-11-27 15:22:33 | INFO     | data_preprocessing:clean_data:81 - Data cleaning completed. Final shape: (8, 11)
2025-11-27 15:22:33 | INFO     | data_preprocessing:create_features:266 - Feature creation completed. New shape: (8, 13)
2025-11-27 15:22:33 | INFO     | data_preprocessing:prepare_features:305 - Numerical features: ['year', 'km_driven', 'mileage', 'engine', 'm

Training set: (6, 16)
Test set: (2, 16)


In [3]:
# Cell 3: Train All Models
trainer = ModelTrainer()

# Train all configured models
models = trainer.train_all_models(X_train, y_train, tune_hyperparameters=False)

print(f"\nTrained {len(models)} models")

2025-11-27 15:22:39 | INFO     | train:__init__:57 - ModelTrainer initialized
2025-11-27 15:22:39 | INFO     | train:train_all_models:76 - Starting training for all configured models
2025-11-27 15:22:39 | INFO     | train:_train_linear_regression:110 - Training Linear Regression model
2025-11-27 15:22:43 | INFO     | train:_train_linear_regression:127 - Linear Regression - CV Score: 95998.3015 (+/- 38983.9983)
2025-11-27 15:22:43 | INFO     | helpers:wrapper:219 - _train_linear_regression executed in 4.0156 seconds
2025-11-27 15:22:43 | INFO     | train:_train_random_forest:149 - Training Random Forest model
2025-11-27 15:22:46 | INFO     | train:_train_random_forest:170 - Random Forest - CV Score: 255725.8887 (+/- 100902.8396)
2025-11-27 15:22:46 | INFO     | helpers:wrapper:219 - _train_random_forest executed in 3.0444 seconds
2025-11-27 15:22:46 | INFO     | train:_train_xgboost:237 - Training XGBoost model
2025-11-27 15:22:48 | INFO     | train:_train_xgboost:258 - XGBoost - CV Sco


Trained 3 models


In [4]:
# Cell 4: Training Summary
summary = trainer.get_training_summary()
print("\nTraining Summary (Cross-Validation RMSE):")
print(summary.to_string(index=False))


Training Summary (Cross-Validation RMSE):
            model  mean_cv_rmse   std_cv_rmse   min_cv_rmse   max_cv_rmse
linear_regression  95998.301512  38983.998321  42406.314854 148305.180532
          xgboost 133667.417187  55669.194302  89733.187500 239981.218750
    random_forest 255725.888700 100902.839645 115900.000000 428500.000000


In [5]:
# Cell 5: Select Best Model
best_name, best_model = trainer.select_best_model(X_test, y_test)
print(f"\nBest Model: {best_name}")

2025-11-27 15:22:56 | INFO     | train:select_best_model:433 - Best model: linear_regression with RMSE: 95998.3015



Best Model: linear_regression


In [6]:
# Cell 6: Feature Importance
feature_names = preprocessor.get_feature_names()
importance_df = trainer.get_feature_importance(model_name=best_name, feature_names=feature_names)

if not importance_df.empty:
    plt.figure(figsize=(10, 8))
    top_n = min(15, len(importance_df))
    plt.barh(importance_df['feature'].head(top_n)[::-1],
             importance_df['importance'].head(top_n)[::-1])
    plt.xlabel('Importance')
    plt.title(f'Top {top_n} Feature Importance ({best_name})')
    plt.tight_layout()
    plt.show()

2025-11-27 15:22:58 | WARNING  | train:get_feature_importance:553 - Model linear_regression does not have feature_importances_


In [7]:
# Cell 7: Save Models
save_path = project_root / 'models'
trainer.save_models(save_all=True, save_path=save_path)
preprocessor.save_preprocessor(save_path / 'preprocessor.pkl')
print(f"\nModels saved to {save_path}")

2025-11-27 15:23:03 | INFO     | helpers:save_pickle:115 - Object saved to D:\practice-1\models\linear_regression_model.pkl
2025-11-27 15:23:03 | INFO     | train:save_models:460 - Saved linear_regression model to D:\practice-1\models\linear_regression_model.pkl
2025-11-27 15:23:04 | INFO     | helpers:save_pickle:115 - Object saved to D:\practice-1\models\random_forest_model.pkl
2025-11-27 15:23:04 | INFO     | train:save_models:460 - Saved random_forest model to D:\practice-1\models\random_forest_model.pkl
2025-11-27 15:23:04 | INFO     | helpers:save_pickle:115 - Object saved to D:\practice-1\models\xgboost_model.pkl
2025-11-27 15:23:04 | INFO     | train:save_models:460 - Saved xgboost model to D:\practice-1\models\xgboost_model.pkl
2025-11-27 15:23:04 | INFO     | helpers:save_pickle:115 - Object saved to D:\practice-1\models\best_model.pkl
2025-11-27 15:23:04 | INFO     | train:save_models:467 - Saved best model (linear_regression) to D:\practice-1\models\best_model.pkl
2025-11-2


Models saved to D:\practice-1\models
